In [264]:
# importing libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

### set location variables for data read-in and saving outputs

In [265]:
# update hospital_extract_data_loc variable with the location of the dataset on your machine
hospital_extract_data_loc = "hospital_extract_20250203.csv"

# update the payer_extract_data_loc variable with the location of the data on your machine
payer_extract_data_loc = "payer_extract_20250203.csv"

# update the output_loc with variable with the location where you would like to save the outputs
output_loc = "C:/Users/Nimra/Downloads/"

#### reading in the data extracts

In [266]:
hp = pd.read_csv(hospital_extract_data_loc)
print(hp.shape)
hp.isna().sum()
hp.head(2)

(222, 17)


,payer,network_name,network_id,network_year_month,network_region,code,code_type,ein,taxonomy_filtered_npi_list,modifier_list,billing_class,place_of_service_list,negotiation_type,arrangement,rate,cms_baseline_schedule,cms_baseline_rate
0,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,872,MS-DRG,131740114,"1003990763,1023202793,1063525152,1063606739,10...",NaN,institutional,NaN,negotiated,ffs,15902.00,IPPS,6829.75
1,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,99283,CPT,131624096,"1003255670,1245759711,1487026522,1598095267,16...",NaN,professional,11,negotiated,ffs,123.86,PFS_NONFACILITY_1320201,76.89


In [267]:
pe = pd.read_csv(payer_extract_data_loc)
print(pe.shape)
pe.isna().sum()
pe.head(2)

(2950, 22)


,source_file_name,hospital_id,hospital_name,last_updated_on,hospital_state,license_number,payer_name,plan_name,code_type,raw_code,description,setting,modifiers,standard_charge_gross,standard_charge_discounted_cash,standard_charge_negotiated_dollar,standard_charge_negotiated_percentage,standard_charge_min,standard_charge_max,standard_charge_methodology,additional_payer_notes,additional_generic_notes
0,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Aetna,Medicare,CPT,99283,EMERGENCY DEPT VISIT LOW MDM,outpatient,NaN,NaN,NaN,323.34,NaN,83.78,1009.22,fee schedule,NaN,NaN
1,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,HealthFirst,Commercial Enrollees,CPT,43239,EGD BIOPSY SINGLE/MULTIPLE,outpatient,NaN,NaN,NaN,1037.65,NaN,165.40,3206.34,fee schedule,NaN,NaN


### some quick exploration of the data that is present

In [268]:
hp['payer'].unique()
hp['network_name'].unique()
hp['network_id'].unique()
hp['network_year_month'].unique()
hp['network_region'].unique()
print(hp['code'].unique())
hp['code_type'].unique()
# hp['taxonomy_filtered_npi_list'].nunique()
# hp['modifier_list'].unique()
# hp['billing_class'].unique()
# hp['negotiation_type'].unique()
# hp['arrangement'].unique()
# hp['cms_baseline_rate'].unique()
# hp['cms_baseline_schedule'].unique()
hp['ein'].unique()

[  872 99283 43239]


array([131740114, 131624096, 133971298], dtype=int64)

#### mapping EINs to hospital names

In [269]:
### some quick googling showed that the eins map as follows
hp['hospital_name'] = ''

hp['hospital_name'].loc[hp['ein']==131740114] = 'Montefiore Medical Center'
hp['hospital_name'].loc[hp['ein']==131624096] = 'The Mount Sinai Hospital'
hp['hospital_name'].loc[hp['ein']==133971298] = 'NYU Langone'

In [270]:
pe['raw_code'].nunique()
pe['raw_code'].unique()
pe['code_type'].unique()
pe['setting'].unique()
pe['source_file_name'].unique()
pe['hospital_name'].unique()

array(['Montefiore Medical Center', 'The Mount Sinai Hospital',
       'NYU Langone'], dtype=object)

### checking for duplicates

In [271]:
# checking for duplicates
print(pe.shape)
pe.drop_duplicates().shape

print(hp.shape)
hp.drop_duplicates().shape

# looking at the 3 duplicated rows in the payer data
pe.loc[pe.duplicated()] # appears to be only for a single hospital

# dropping duplicates for now since only 3 rows out of ~3K
pe = pe.drop_duplicates().reset_index(drop=True)
print(pe.shape)

(2950, 22)
(222, 18)
(2947, 22)


### standardizing codes

In [272]:
# standardizing the column names between the two extracts for convenience, arbitrarily picked "code" for column name here 
# since it's simpler here
pe.rename(columns=({'raw_code': 'code'}), inplace=True)
pe.head(3)

# checking if MS-DRG 872 is the same as 872 in this dataset i.e. always occur together
test = pe.loc[pe['code']=='872']
print(test.shape)
print(test['code_type'].unique()) # appears true for the payer data extract

# doing the same for the hospital dataset
hp['code'].unique()
test = hp.loc[hp['code']==872]
print(test.shape)
print(test['code_type'].unique()) # also true for the hospital data extract

# making the code columns the same data type, saving as integers for now since takes less space in Python. 
# the prefix MS-DRG does contain information so likely want to maintain that info somewhere (maintained in code_type here)
print(pe['code'].unique())
pe['code'].loc[pe['code']=='MS-DRG 872'] = 872
pe['code'] = pe['code'].astype('int64')

# # confirming that changes are present
pe['code'].unique()

(88, 22)
['MS-DRG']
(19, 18)
['MS-DRG']
['99283' '43239' '872' 'MS-DRG 872']


array([99283, 43239,   872], dtype=int64)

### some very quick standardization of payer names

In [273]:
# doing some very basic manual string standardization here, since we only have 3 payers in the hospital dataset. 
# Ideally would want to build up and maintain a reference list over time of the different names 
# that have been / can be used for the same payer for quick and efficient cross-matching.

print(pe['payer_name'].unique())
print(hp['payer'].unique())

# standardizing column name across datasets
pe.rename(columns=({'payer_name': 'payer'}), inplace=True)
pe['payer'].loc[pe['payer']=='cigna'] = 'Cigna'
pe['payer'].loc[pe['payer']=='aetna'] = 'Aetna'

# standardizing column values for the 3 plans in the hospital data, used snake case here bc more readable
hp['payer'].loc[hp['payer']=='unitedhealthcare'] = 'United Healthcare'
hp['payer'].loc[hp['payer']=='aetna'] = 'Aetna'
hp['payer'].loc[hp['payer']=='cigna-corporation'] = 'Cigna'

# have 2 entries in payer extract that may be a typo of United Healthcare, checking freq of occurence
pe.loc[pe['payer'].isin(['United', 'Healthcare'])].shape
# few enough that may be a typo, leaving alone for now until can research further

['Aetna' 'HealthFirst' 'Cigna' 'Oscar' 'Healthcare' 'United' 'Emblem'
 'Humana' 'Fidelis' 'Centerlight' 'Partners' 'Northwell' 'Agewell'
 'MetroPlus' 'Longevity' 'MVP' 'Hamaspik' 'WellCare' 'Empire' 'Elderplan'
 'VNSNY' 'Nat' 'CentersPlan' 'UHC' 'VillageCareMAX' 'Senior' 'Nippon'
 'Wellcare' 'SEIU1199' 'Molina' 'Multiplan' 'Magnacare' 'Centivo' 'Beacon'
 'United Healthcare' 'Bright Health' 'Horizon' 'Amida Care' 'Oxford'
 'Empire Medicare Advantage' 'Lifetrac' 'Independence Care' 'Affinity'
 '1199' 'bcbs' 'multiplan' 'cigna' 'oxford' 'hip' 'medicare' 'uhc' 'aetna'
 'somos' 'metroplus' 'healthplus' 'healthfirst' 'fidelis' 'ghi'
 'magnacare' 'amida' 'firsthealth' 'wellcare' 'medicaid' 'threerivers'
 'centerlight' 'hamaspikchoice' 'Amidacare' 'Archcare' 'Christian'
 'American' 'Brighton Health' 'Empire Healthplus' 'VNS']
['unitedhealthcare' 'aetna' 'cigna-corporation']


(18, 22)

### look at dates for potential matching signal

In [274]:
print(pe['last_updated_on'].unique())
print(hp['network_year_month'].unique())

# if this data is updated every month (as opposed to only when needed)
# may be able to keep only the 01-2025 rates to match against the hospital data

# checking if data is duplicated across months in the payer data
cols_to_check = [x for x in pe.columns if x not in ['last_updated_on']]
print(cols_to_check)
pe.duplicated(subset=cols_to_check).shape # doesn't seem like it

# keeping all months of "last_updated_on" in payer data, since inconclusive for now

['2024-07-01' '2024-09-16' '2025-01-01']
[202501]
['source_file_name', 'hospital_id', 'hospital_name', 'hospital_state', 'license_number', 'payer', 'plan_name', 'code_type', 'code', 'description', 'setting', 'modifiers', 'standard_charge_gross', 'standard_charge_discounted_cash', 'standard_charge_negotiated_dollar', 'standard_charge_negotiated_percentage', 'standard_charge_min', 'standard_charge_max', 'standard_charge_methodology', 'additional_payer_notes', 'additional_generic_notes']


(2947,)

### checking to see if there are additional columns that can be matched on

In [275]:
print(hp['cms_baseline_schedule'].unique())
print(hp['billing_class'].unique())
print(hp['arrangement'].unique())
print(hp['negotiation_type'].unique())
# print(hp['place_of_service_list'].unique())

# try to find comparable flag in payer data extract
print(' ')
print("PAYER INFO")
print(pe['setting'].unique())
# print(pe['description'].unique())
print(pe['standard_charge_methodology'].unique())
# print(pe['additional_payer_notes'].unique())

['IPPS' 'PFS_NONFACILITY_1320201' 'PFS_NONFACILITY_1320202'
 'PFS_NONFACILITY_1320203' 'PFS_NONFACILITY_0610216' 'OPPS'
 'PFS_NONFACILITY_1240201' 'PFS_FACILITY_1320202'
 'PFS_NONFACILITY_1329204' 'PFS_FACILITY_NPA' 'PFS_NONFACILITY_0111263'
 'PFS_FACILITY_1320201' nan 'PFS_NONFACILITY_1328299'
 'PFS_NONFACILITY_0820201' 'PFS_FACILITY_1329204' 'PFS_FACILITY_1240201'
 'PFS_FACILITY_0111263' 'PFS_NONFACILITY_0910299']
['institutional' 'professional']
['ffs']
['negotiated' 'fee schedule' 'percentage']
 
PAYER INFO
['outpatient' 'both' 'inpatient']
['fee schedule' 'other' 'case rate' 'per diem' 'Fee Schedule'
 'percent of total billed charges' 'Other' 'Case Rate' nan
 'Percent of total billed charges' 'Case rate' '5228' '93157.15' '11671'
 '32829.5' '17412.69']


### check matching on the 3 columns we have the most confidence in / are the most straightforward: code, payer, hospital_name

In [276]:
# keep only the payers that are in the hospital dataset for now
pe_subset = pe.loc[pe['payer'].isin(['Aetna', 'United Healthcare', 'Cigna'])].reset_index(drop=True)
print(pe_subset.shape)

merged_df = hp.merge(pe_subset, on=['code', 'payer', 'hospital_name'], how='inner', indicator=True)
print(("hospital extract # rows:", hp.shape[0]))
print(("payer extract # rows:", pe_subset.shape[0]))
print(merged_df.shape) # 

(265, 22)
('hospital extract # rows:', 222)
('payer extract # rows:', 265)
(2857, 38)


### checking match rates

In [277]:
# checking match rate
temp = hp.merge(pe_subset[['code', 'payer', 'hospital_name']], on=['code', 'payer', 'hospital_name'],
                                         how='left', indicator=True)
print(temp.shape)
temp.head()

unmatched_hospital = temp.loc[temp['_merge']=='left_only'].reset_index(drop=True)
print(unmatched_hospital.shape)
# saving unmatched rows
unmatched_hospital.drop(columns=['_merge'], inplace=True)
unmatched_hospital = unmatched_hospital.drop_duplicates().reset_index(drop=True)
file_loc = output_loc + "hospital_extract_unmatched_rows.csv"
unmatched_hospital.to_csv(file_loc, index=False)
print(f"{unmatched_hospital.shape[0]} rows in the hospital data were not matched out of {hp.shape[0]} total rows in hospital extract")
match_rate = round((hp.shape[0] - unmatched_hospital.shape[0]) / hp.shape[0] * 100)
print(f"match rate: {match_rate}%")

# repeating above for the payer extract
temp = pe_subset.merge(hp[['code', 'payer', 'hospital_name']], on=['code', 'payer', 'hospital_name'],
                                         how='left', indicator=True)
print(temp.shape)
temp.head()

unmatched_payer = temp.loc[temp['_merge']=='left_only'].reset_index(drop=True)
print(unmatched_payer.shape)
# saving unmatched rows
unmatched_payer.drop(columns=['_merge'], inplace=True)
unmatched_payer = unmatched_payer.drop_duplicates().reset_index(drop=True)
file_loc = output_loc + "payer_extract_unmatched_rows.csv"
unmatched_payer.to_csv(file_loc, index=False)
print(f"{unmatched_payer.shape[0]} rows in the payer data were not matched out of {pe_subset.shape[0]} total rows (for relevant payers only) in the payer extract")
match_rate = round((pe_subset.shape[0] - unmatched_payer.shape[0]) / pe_subset.shape[0] * 100)
print(f"match rate: {match_rate}%")

(2890, 19)
(33, 19)
33 rows in the hospital data were not matched out of 222 total rows in hospital extract
match rate: 85%
(2878, 23)
(21, 23)
21 rows in the payer data were not matched out of 265 total rows (for relevant payers only) in the payer extract
match rate: 92%


### delving further into potential columns that may be used for matching

#### looks like "setting" in the payer extract may be analogous to "cms_baseline_schedule" in hospital extract

In [278]:
hp['cms_baseline_schedule'].unique()

array(['IPPS', 'PFS_NONFACILITY_1320201', 'PFS_NONFACILITY_1320202',
       'PFS_NONFACILITY_1320203', 'PFS_NONFACILITY_0610216', 'OPPS',
       'PFS_NONFACILITY_1240201', 'PFS_FACILITY_1320202',
       'PFS_NONFACILITY_1329204', 'PFS_FACILITY_NPA',
       'PFS_NONFACILITY_0111263', 'PFS_FACILITY_1320201', nan,
       'PFS_NONFACILITY_1328299', 'PFS_NONFACILITY_0820201',
       'PFS_FACILITY_1329204', 'PFS_FACILITY_1240201',
       'PFS_FACILITY_0111263', 'PFS_NONFACILITY_0910299'], dtype=object)

In [279]:
pe['setting']

0       outpatient
1       outpatient
2       outpatient
3       outpatient
4       outpatient
           ...    
2942          both
2943     inpatient
2944          both
2945     inpatient
2946          both
Name: setting, Length: 2947, dtype: object

In [280]:
# create a new column for 'setting' in the hospital data corresponding to the 'setting column
# in the payer data, based on the cms_baseline_schedule column in the hospital data
# choosing to create setting to match since it is the broader/ more general category

hp['cms_baseline_schedule'].loc[hp['cms_baseline_schedule'].isna()] = 'unknown' # deal with missing vals 

hp['setting'] = np.nan
hp['setting'].loc[hp['cms_baseline_schedule'].isin(['IPPS'])] = 'inpatient'
hp['setting'].loc[hp['cms_baseline_schedule'].isin(['OPPS', 'PFS_NONFACILITY_'])] = 'outpatient'
hp['setting'].loc[hp['cms_baseline_schedule'].str.contains('PFS_NONFACILITY_')] = 'outpatient'


# less sure about this mapping but keeping for now
hp['setting'].loc[hp['cms_baseline_schedule'].str.contains('PFS_FACILITY_')] = 'both'

# some checks to see if above occurred correctly
hp.isna().sum()
test = hp.loc[hp['setting'].isna()]
test['cms_baseline_schedule'].unique()
test

,payer,network_name,network_id,network_year_month,network_region,code,code_type,ein,taxonomy_filtered_npi_list,modifier_list,billing_class,place_of_service_list,negotiation_type,arrangement,rate,cms_baseline_schedule,cms_baseline_rate,hospital_name,setting
84,Aetna,open-access-managed-choice,39f0d406-b5df-4046-9759-f08565e45db7,202501,USA,99283,CPT,131740114,"1063525152,1235584079,1306423975,1851151971",NaN,professional,23,negotiated,ffs,267.0,unknown,NaN,Montefiore Medical Center,NaN


#### checking if negotiation_type in hospital data can be analogous to standard_charge_methodology in payer data

In [281]:
# seeing if can match on arrangement / negotiation type in hospital data to 'standard_charge_methododlogy
# in the payer data

print(hp['arrangement'].unique()) ## all values say ffs so less informative on its own
print(hp['negotiation_type'].unique())

print(' ')
print('PAYER INFO')
print(pe['standard_charge_methodology'].unique())

# standardize values in the standard_charge_methodology_column / fill in missing values
pe['standard_charge_methodology'].loc[pe['standard_charge_methodology'].isin(['Fee Schedule', 
                                                                              'fee schedule'])] = 'fee schedule'
pe['standard_charge_methodology'].loc[pe['standard_charge_methodology'].isin(['other', 
                                                                              'Other'])] = 'other'
pe['standard_charge_methodology'].loc[pe['standard_charge_methodology'].isin(['case rate', 
                                                                   'Case Rate', 
                                                                   'Case rate'])] = 'case rate'
pe['standard_charge_methodology'].loc[pe['standard_charge_methodology'].isna()] = 'unknown'
pe['standard_charge_methodology'].loc[pe['standard_charge_methodology'].isin(['Percent of total billed charges'])] = 'percent of total billed charges'

# create an additional column to match to in the hospital data (negotiation_type) and fill out
pe['negotiation_type'] = np.nan
pe['negotiation_type'].loc[pe['standard_charge_methodology']=='percent of total billed charges'] = 'percentage'
pe['negotiation_type'].loc[pe['standard_charge_methodology']=='fee schedule'] = 'fee schedule'
pe['negotiation_type'].loc[pe['standard_charge_methodology'].isin(['other', 'case rate', 'unknown',
                                                                   'per diem',
                                                                   '5228', '93157.15', '11671', '32829.5', '17412.69'])] = 'negotiated'
pe['negotiation_type'].loc[pe['standard_charge_methodology']=='unknown'] = 'unknown'


['ffs']
['negotiated' 'fee schedule' 'percentage']
 
PAYER INFO
['fee schedule' 'other' 'case rate' 'per diem' 'Fee Schedule'
 'percent of total billed charges' 'Other' 'Case Rate' nan
 'Percent of total billed charges' 'Case rate' '5228' '93157.15' '11671'
 '32829.5' '17412.69']


In [282]:
# quick check of changes made above 
print(pe['negotiation_type'].unique())
print(hp['negotiation_type'].unique())

['fee schedule' 'negotiated' 'percentage' 'unknown']
['negotiated' 'fee schedule' 'percentage']


#### looking further into rates, if can match directly on them

In [283]:
hp.columns
hp.head(2)

# hospital data is relatively straightforward, only 2 rates mentioned (rate, cms_baseline_rate)

pe.isna().sum()
# have a large # of nans in the various rates column in the payer data so checking further
pe['nan_count'] = pe[['standard_charge_gross', 'standard_charge_discounted_cash', 
                      'standard_charge_negotiated_dollar', 'standard_charge_negotiated_percentage',
                      'standard_charge_min','standard_charge_max']].isna().sum(axis=1)
print(pe['nan_count'].unique()) # looks like have at least one rate for each row in payer data

# min and max appear to be the most populated columns, don't have a direct comparison in hospital data

# if the rates are mutually exclusive, for e.g. if 'standard_charge_negotiated_percentage' is populated,
# then does 'standard_charge_negotiated_dollar' always have to be nan and vice versa

[3 1 2 5 4]


### matching the data now using the additional columns standardized above

In [284]:
# keep only the payers that are in the hospital dataset, with the additional columns above
pe_subset = pe.loc[pe['payer'].isin(['Aetna', 'United Healthcare', 'Cigna'])].reset_index(drop=True)
print(pe_subset.shape)


merged_df = hp.merge(pe_subset, on=['code', 'payer', 'hospital_name', 
                                    'setting', 'negotiation_type'], how='inner', indicator=True)
print(merged_df.shape)

(265, 24)
(434, 39)


#### calculating match rate with additional columns used

In [285]:
# checking match rate
temp = hp.merge(pe_subset[['code', 'payer', 'hospital_name', 'setting', 'negotiation_type']],
                on=['code', 'payer', 'hospital_name', 'setting', 'negotiation_type'],
                how='left', indicator=True)
print(temp.shape)
temp.head()

unmatched_hospital = temp.loc[temp['_merge']=='left_only'].reset_index(drop=True)
print(unmatched_hospital.shape)
# calculating unmatched rows
unmatched_hospital.drop(columns=['_merge'], inplace=True)
unmatched_hospital = unmatched_hospital.drop_duplicates().reset_index(drop=True)
print(f"{unmatched_hospital.shape[0]} rows in the hospital data were not matched out of {hp.shape[0]} total rows in hospital extract")
match_rate = round((hp.shape[0] - unmatched_hospital.shape[0]) / hp.shape[0] * 100)
print(f"match rate: {match_rate}%")

# repeating above for the payer extract
temp = pe_subset.merge(hp[['code', 'payer', 'hospital_name', 'setting', 'negotiation_type']], 
                       on=['code', 'payer', 'hospital_name', 'setting', 'negotiation_type'],
                       how='left', indicator=True)
print(temp.shape)
temp.head()

unmatched_payer = temp.loc[temp['_merge']=='left_only'].reset_index(drop=True)
print(unmatched_payer.shape)
# calculating unmatched rows
unmatched_payer.drop(columns=['_merge'], inplace=True)
unmatched_payer = unmatched_payer.drop_duplicates().reset_index(drop=True)
print(f"{unmatched_payer.shape[0]} rows in the payer data were not matched out of {pe_subset.shape[0]} total rows (for relevant payers only) in the payer extract")
match_rate = round((pe_subset.shape[0] - unmatched_payer.shape[0]) / pe_subset.shape[0] * 100)
print(f"match rate: {match_rate}%")

(599, 20)
(165, 20)
165 rows in the hospital data were not matched out of 222 total rows in hospital extract
match rate: 26%
(606, 25)
(172, 25)
172 rows in the payer data were not matched out of 265 total rows (for relevant payers only) in the payer extract
match rate: 35%


#### see data "expansion" when go down sharply but unmatched data also increases substantially

### want to test if any direct matches on rate

In [286]:
pe_subset.isna().sum()
# standard_charge_negotiated_dollar is the most populated rate column in the payer data

pe_subset.loc[pe_subset['standard_charge_negotiated_dollar'].isna()]

,source_file_name,hospital_id,hospital_name,last_updated_on,hospital_state,license_number,payer,plan_name,code_type,code,description,setting,modifiers,standard_charge_gross,standard_charge_discounted_cash,standard_charge_negotiated_dollar,standard_charge_negotiated_percentage,standard_charge_min,standard_charge_max,standard_charge_methodology,additional_payer_notes,additional_generic_notes,negotiation_type,nan_count
226,133971298-1801992631_nyu-langone-tisch_standar...,40e6a8c8-a68c-4d28-b1d5-fa70d6d09636,NYU Langone,2025-01-01,NY,7002053H,Aetna,aetnaindemnity1006,CPT,43239,EGD BIOPSY SINGLE/MULTIPLE,both,NaN,NaN,NaN,NaN,100.0,NaN,NaN,percent of total billed charges,100 percent of billed charges,NaN,percentage,5
249,133971298-1801992631_nyu-langone-tisch_standar...,40e6a8c8-a68c-4d28-b1d5-fa70d6d09636,NYU Langone,2025-01-01,NY,7002053H,Aetna,healthscope3250,LOCAL,43239,HEAD HUM 19MM 50MM SHLDR UNIVERS STRL LF CUF A...,both,NaN,32829.5,6237.61,NaN,NaN,NaN,NaN,unknown,Not separately priced,NaN,unknown,4


In [287]:
pe_subset['match_flag'] = False
rate_cols = ['standard_charge_gross', 
             'standard_charge_discounted_cash', 'standard_charge_negotiated_dollar', 
             'standard_charge_negotiated_percentage', 'standard_charge_min',
             'standard_charge_max']

# checking if any rate matches across the data - this code is inefficient but works since our sample is small, 
# would minimize for loops use depending on size of the data
for p in ['United Healthcare', 'Aetna', 'Cigna']:
    for code in [99283, 43239, 872]:
        subset = hp.loc[((hp['payer']==p) & (hp['code']==code))]
        rates_to_match = subset['rate'].unique().tolist()
        # print(rates_to_match)
        pe_subset['match_flag'] = pe_subset[rate_cols].isin(rates_to_match).any(axis=1)

In [288]:
pe_subset.loc[pe_subset['match_flag']==True].shape # looks like no direct matches on rate across codes

(0, 25)

## Final Outputs Prepared

In [289]:
merged_df = hp.merge(pe_subset, on=['code', 'payer', 'hospital_name'], how='inner', indicator=True)
print(("hospital extract # rows:", hp.shape[0]))
print(("payer extract # rows:", pe_subset.shape[0]))
print(merged_df.shape) # 

('hospital extract # rows:', 222)
('payer extract # rows:', 265)
(2857, 42)


### keeping only key columns / clean-up of columns names for the final output

In [290]:
cols_to_use = ['payer', 'code', 'hospital_name','code_type_x',
               'hospital_state', 'cms_baseline_rate', 'rate','standard_charge_gross', 'standard_charge_discounted_cash', 
               'standard_charge_negotiated_dollar', 'standard_charge_negotiated_percentage', 'standard_charge_min',
               'standard_charge_max', 'standard_charge_methodology']

output_df = merged_df[cols_to_use]
output_df.rename(columns=({'code_type_x': 'code_type', 
                           'rate': 'hospital_rate',
                           'standard_charge_gross': 'payer_standard_charge_gross', 
                           'standard_charge_discounted_cash': 'payer_standard_charge_discounted_cash', 
                           'standard_charge_negotiated_dollar': 'payer_standard_charge_negotiated_dollar', 
                           'standard_charge_negotiated_percentage': 'payer_standard_charge_negotiated_percentage', 
                           'standard_charge_min': 'payer_standard_charge_min',
                           'standard_charge_max': 'payer_standard_charge_max', 
                           'standard_charge_methodology': 'payer_standard_charge_methodology'}), inplace=True)

print(output_df.shape)
output_df.head(2)

(2857, 14)


,payer,code,hospital_name,code_type,hospital_state,cms_baseline_rate,hospital_rate,payer_standard_charge_gross,payer_standard_charge_discounted_cash,payer_standard_charge_negotiated_dollar,payer_standard_charge_negotiated_percentage,payer_standard_charge_min,payer_standard_charge_max,payer_standard_charge_methodology
0,United Healthcare,99283,The Mount Sinai Hospital,CPT,NY,76.89,123.86,2100.0,329.95,329.95,NaN,78.0,395.94,fee schedule
1,United Healthcare,99283,The Mount Sinai Hospital,CPT,NY,76.89,123.86,2100.0,329.95,329.95,NaN,78.0,395.94,fee schedule


#### saving the output merged df

In [291]:
file_loc = output_loc + "matched_data.csv"
output_df.to_csv(file_loc)

In [292]:
temp = hp.merge(pe_subset[['code', 'payer', 'hospital_name']], on=['code', 'payer', 'hospital_name'],
                                         how='left', indicator=True)
print(temp.shape)
temp.head()

unmatched_hospital = temp.loc[temp['_merge']=='left_only'].reset_index(drop=True)
print(unmatched_hospital.shape)
# saving unmatched rows
unmatched_hospital.drop(columns=['_merge'], inplace=True)
unmatched_hospital = unmatched_hospital.drop_duplicates().reset_index(drop=True)
file_loc = output_loc + "hospital_extract_unmatched_rows.csv"
unmatched_hospital.to_csv(file_loc, index=False)
print(f"{unmatched_hospital.shape[0]} rows in the hospital data were not matched out of {hp.shape[0]} total rows in hospital extract")

# repeating above for the payer extract
temp = pe_subset.merge(hp[['code', 'payer', 'hospital_name']], on=['code', 'payer', 'hospital_name'],
                                         how='left', indicator=True)
print(temp.shape)
temp.head()

unmatched_payer = temp.loc[temp['_merge']=='left_only'].reset_index(drop=True)
print(unmatched_payer.shape)
# saving unmatched rows
unmatched_payer.drop(columns=['_merge'], inplace=True)
unmatched_payer = unmatched_payer.drop_duplicates().reset_index(drop=True)
file_loc = output_loc + "payer_extract_unmatched_rows.csv"
unmatched_payer.to_csv(file_loc, index=False)
print(f"{unmatched_payer.shape[0]} rows in the payer data were not matched out of {pe_subset.shape[0]} total rows (for relevant payers only) in the payer extract")

(2890, 20)
(33, 20)
33 rows in the hospital data were not matched out of 222 total rows in hospital extract
(2878, 26)
(21, 26)
21 rows in the payer data were not matched out of 265 total rows (for relevant payers only) in the payer extract
